# Test batch processing with Delta

This is a notebook to test the delta_batch_process script. 

Run this notebook to make sure you can read your files.

Then process them using the equivalent script `delta_batch_process.py`. 

You can run this on a cluster by submitting `slurm_delta_batch.sh` using sbatch.



In [1]:

import pathlib
import delta.config as cfg
from delta.utilities import xpreader
from delta.pipeline import Pipeline
import tensorflow as tf


In [2]:
dev = tf.config.list_physical_devices()
print(dev)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


2022-06-23 08:33:49.238558: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /scicore/home/vanvli0000/vanvli0000/mambaforge/envs/delta2_env/lib/python3.9/site-packages/cv2/../../lib64:/scicore/home/vanvli0000/vanvli0000/mambaforge/envs/delta2_env/lib/
2022-06-23 08:33:49.238668: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-23 08:33:49.238725: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (shi125.cluster.bc2.ch): /proc/driver/nvidia/version does not exist


In [3]:

def to_str(posixpath):
    return str(posixpath.resolve())   


In [5]:

#set paths
root = pathlib.Path(pathlib.Path.home(), 'home', 'Delta2_Caulobacter')
data_dir = root / 'data' / 'rawdata'

#create output dir
output_root = root / 'processed_data'
(output_root).mkdir(exist_ok=True) #create output data folder,  each position will be placed in a subfolder

#find subfolders
folder_names = [f.name for f in sorted(data_dir.glob('AKS*'))]

for f in folder_names: print(f)

AKS1088
AKS1089
AKS1090
AKS1091
AKS1092
AKS1093
AKS1094
AKS1095
AKS1096
AKS1097
AKS1098
AKS1099
AKS1101
AKS1102
AKS1103
AKS1104
AKS1105
AKS1106


In [17]:
print(output_root)

/scicore/home/vanvli0000/vanvli0000/home/Delta2_Caulobacter/processed_data


In [7]:
#get config file
#make sure you adapt the model paths in the json file!
config_file = root / 'config_2D_caulobacter.json'
cfg.load_config(config_file)
cfg.save_format = ('pickle','movie')

Loading configuration from: /scicore/home/vanvli0000/vanvli0000/home/Delta2_Caulobacter/config_2D_caulobacter.json


In [8]:
folder = folder_names[0]
movie_names = [f.name for f in sorted((data_dir / folder).glob('*.tif*'))]
print(movie_names)

['20220311_AKS1088-1093_dense_TL01_01_R3D-1.tif', '20220311_AKS1088-1093_dense_TL01_02_R3D-1.tif', '20220311_AKS1088-1093_dense_TL01_03_R3D-1.tif', '20220311_AKS1088-1093_dense_TL01_04_R3D-1.tif', '20220311_AKS1088-1093_dense_TL01_05_R3D-1.tif']


In [20]:
movie = movie_names[0]
#path to current position
movie_dir = data_dir / folder / movie



TL01_01


In [13]:
#make subfolder for current position
#create output subfolder
output_path = output_root / folder
(output_path).mkdir(exist_ok=True) #create output data folder,  each position will be placed in a subfolder
    
output_dir = output_path / movie
(output_dir).mkdir(exist_ok=True)

im_reader = xpreader(movie_dir, use_bioformats=True)
# Print experiment parameters to make sure it initialized properly:
print("""Initialized experiment reader:
    - %d positions
    - %d imaging channels
    - %d timepoints"""%(im_reader.positions, im_reader.channels, im_reader.timepoints)
)
    

08:44:03.389 [Thread-0] DEBUG loci.formats.ClassList - Could not find loci.formats.in.URLReader
java.lang.ClassNotFoundException: loci.formats.in.URLReader
	at java.base/jdk.internal.loader.BuiltinClassLoader.loadClass(BuiltinClassLoader.java:583) ~[na:na]
	at java.base/jdk.internal.loader.ClassLoaders$AppClassLoader.loadClass(ClassLoaders.java:178) ~[na:na]
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:521) ~[na:na]
	at java.base/java.lang.Class.forName0(Native Method) ~[na:na]
	at java.base/java.lang.Class.forName(Class.java:315) ~[na:na]
	at loci.formats.ClassList.parseLine(ClassList.java:196) ~[bioformats_package.jar:6.6.0]
	at loci.formats.ClassList.parseFile(ClassList.java:258) ~[bioformats_package.jar:6.6.0]
	at loci.formats.ClassList.<init>(ClassList.java:138) ~[bioformats_package.jar:6.6.0]
08:44:03.390 [Thread-0] DEBUG loci.formats.ClassList - Could not find loci.formats.in.SlideBook6Reader
java.lang.ClassNotFoundException: loci.formats.in.SlideBook6Reader
	a

In [14]:
xp = Pipeline(im_reader, resfolder=to_str(output_dir))  

2022-06-23 08:44:07.894166: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
xp.process(frames=list(range(4)))

In [12]:
#ignore this 

for folder in folder_names:
    
    #get images in subfolder
    movie_names = [f.name for f in sorted((data_dir / folder).glob('*.tif*'))]

    #create output subfolder
    output_path = output_root / folder
    (output_path).mkdir(exist_ok=True) #create output data folder,  each position will be placed in a subfolder

    for movie in movie_names:        
        #path to current position
        movie_dir = data_dir / folder / movie
        
        #make nickname of movei (adapt to file name structure, here we take the part starting at TL and stopping before __R3D)
        start = movie.find('TL')
        end = movie.find('_R3D', start)
        movie_name_short = movie[start:end]
        
        #make subfolder for current position
        output_dir = output_path / movie_name_short
        (output_dir).mkdir(exist_ok=True)

        try:  
            print('starting with movie %s->%s' %(folder,movie_name_short)) 
            # Init reader (use bioformats=True if working with nd2, czi, ome-tiff etc):
            im_reader = xpreader(movie_dir, use_bioformats=True)

            # Print experiment parameters to make sure it initialized properly:
            print("""Initialized experiment reader:
                - %d positions
                - %d imaging channels
                - %d timepoints"""%(im_reader.positions, im_reader.channels, im_reader.timepoints)
            )

            # Init pipeline:
            xp = Pipeline(im_reader, resfolder=to_str(output_dir))   

            # Run it (you can specify which positions, which frames to run etc):
            # we skip running the pipeline and only test IO
            # xp.process()
            
        except:
            print('error with movie %s->%s, skipping to next' %(folder,movie_name_short)) 


starting with position  20220311_AKS1088-1093_dense_TL01_01_R3D-1.tif
skipping postion 20220311_AKS1088-1093_dense_TL01_01_R3D-1.tif
starting with position  20220311_AKS1088-1093_dense_TL01_02_R3D-1.tif
skipping postion 20220311_AKS1088-1093_dense_TL01_02_R3D-1.tif
starting with position  20220311_AKS1088-1093_dense_TL01_03_R3D-1.tif
skipping postion 20220311_AKS1088-1093_dense_TL01_03_R3D-1.tif
starting with position  20220311_AKS1088-1093_dense_TL01_04_R3D-1.tif
skipping postion 20220311_AKS1088-1093_dense_TL01_04_R3D-1.tif
starting with position  20220311_AKS1088-1093_dense_TL01_05_R3D-1.tif
skipping postion 20220311_AKS1088-1093_dense_TL01_05_R3D-1.tif
